# <font class='ign_color'>ROS IN 5 DAYS</font>

# Unit 4: ROS Actions

<img src="img/drone.jpg" width="450" />

<b>Estimated time to completion:</b> 2.5 hours
<br><br>
<b>What will you learn with this unit?</b>

* How to create an action server
* How to build your own action message

## Part 2

In the previous lesson, you learned how to <b>call</b> an action server creating an action client. In this lesson, you are going to learn how to <b>create</b> your own action server.

<figure>
  <img id="fig-4.5" src="img/action_interface.png"/>
  <br>
   <center> <figcaption>Fig.4.5 - Action Interface Diagram Copy 2</figcaption></center>
</figure>

## Writing an action server

<p style="background:#EE9023;color:white;">**Exercise 4.11: Test Fibonacci Action Server through Notebook**</p>

* Create a new package named **action_server_pkg**. When creating the package, add as dependencies **roscpp**, **actionlib** and **actionlib_msgs**.

* Inside the src folder of the package, create a new file named **fibonacci_action_server.cpp**. Inside this file, copy the contents of <a href="#prg-4.11a">fibonacci_action_server.cpp</a>.

* Create a launch file for launching this code.

* Do the necessary modifications to your **CMakeLists.txt** file, and compile the package.

* Execute the launch file to run your executable.

The code below is an example of a ROS action server. When called, the action server will generate a Fibonacci sequence of a given order. The action server goal message must indicate the order of the sequence to be calculated, the feedback of the sequence as it is being computed, and the result of the final Fibonacci sequence.

<p style="background:#EE9023;color:white;">**END Exercise 4.11**</p>

<p style="background:#3B8F10;color:white;" id="prg-4.11a">**C++ Program {4.11a}: fibonacci_action_server.cpp** </p>

In [ ]:
#include <ros/ros.h>
#include <actionlib/server/simple_action_server.h>
#include <actionlib_tutorials/FibonacciAction.h>

class FibonacciAction
{
protected:

  ros::NodeHandle nh_;
  // NodeHandle instance must be created before this line. Otherwise strange error occurs.
  actionlib::SimpleActionServer<actionlib_tutorials::FibonacciAction> as_; 
  std::string action_name_;
  // create messages that are used to publish feedback and result
  actionlib_tutorials::FibonacciFeedback feedback_;
  actionlib_tutorials::FibonacciResult result_;

public:

  FibonacciAction(std::string name) :
    as_(nh_, name, boost::bind(&FibonacciAction::executeCB, this, _1), false),
    action_name_(name)
  {
    as_.start();
  }

  ~FibonacciAction(void)
  {
  }

  void executeCB(const actionlib_tutorials::FibonacciGoalConstPtr &goal)
  {
    // helper variables
    ros::Rate r(1);
    bool success = true;

    // push_back the seeds for the fibonacci sequence
    feedback_.sequence.clear();
    feedback_.sequence.push_back(0);
    feedback_.sequence.push_back(1);

    // publish info to the console for the user
    ROS_INFO("%s: Executing, creating fibonacci sequence of order %i with seeds %i, %i", action_name_.c_str(), goal->order, feedback_.sequence[0], feedback_.sequence[1]);

    // start executing the action
    for(int i=1; i<=goal->order; i++)
    {
      // check that preempt has not been requested by the client
      if (as_.isPreemptRequested() || !ros::ok())
      {
        ROS_INFO("%s: Preempted", action_name_.c_str());
        // set the action state to preempted
        as_.setPreempted();
        success = false;
        break;
      }
      feedback_.sequence.push_back(feedback_.sequence[i] + feedback_.sequence[i-1]);
      // publish the feedback
      as_.publishFeedback(feedback_);
      // this sleep is not necessary, the sequence is computed at 1 Hz for demonstration purposes
      r.sleep();
    }

    if(success)
    {
      result_.sequence = feedback_.sequence;
      ROS_INFO("%s: Succeeded", action_name_.c_str());
      // set the action state to succeeded
      as_.setSucceeded(result_);
    }
  }


};


int main(int argc, char** argv)
{
  ros::init(argc, argv, "fibonacci");

  FibonacciAction fibonacci("fibonacci");
  ros::spin();

  return 0;
}

<p style="background:#3B8F10;color:white;">**Code Explanation C++ Program: {4.11a}**</p>

In this case, the action server is using an action message definition called <i>Fibonacci.action</i>. That message has been created by ROS into its <i>actionlib_tutorials</i> package.

In [ ]:
protected:

  ros::NodeHandle nh_;
  // NodeHandle instance must be created before this line. Otherwise strange error occurs.
  actionlib::SimpleActionServer<actionlib_tutorials::FibonacciAction> as_;
  std::string action_name_;
  // create messages that are used to published feedback/result
  actionlib_tutorials::FibonacciFeedback feedback_;
  actionlib_tutorials::FibonacciResult result_;

These are the protected variables of the action class. The node handle is constructed and passed into the action server during construction of the action. The action server is constructed in the constructor of the action and has been discussed below. The feedback and result messages are created for publishing in the action.

In [ ]:
FibonacciAction(std::string name) :
  as_(nh_, name, boost::bind(&FibonacciAction::executeCB, this, _1), false),
  action_name_(name)
{
  as_.start();
}

In the class constructor, an action server is created. The action server takes arguments of a node handle, name of the action, and optionally an executeCB. In this example the action server is created with the arguments for the executeCB.

In [ ]:
void executeCB(const actionlib_tutorials::FibonacciGoalConstPtr &goal)
{

Now the executeCB function referenced in the constructor is created. The callback function is passed a pointer to the goal message. **Note**: This is a boost shared pointer, given by appending "ConstPtr" to the end of the goal message type.

In [ ]:
    ros::Rate r(1);
    bool success = true;

    // push_back the seeds for the fibonacci sequence
    feedback_.sequence.clear();
    feedback_.sequence.push_back(0);
    feedback_.sequence.push_back(1);

    // publish info to the console for the user
    ROS_INFO("%s: Executing, creating fibonacci sequence of order %i with seeds %i, %i", action_name_.c_str(), goal->order, feedback_.sequence[0], feedback_.sequence[1]);

Here the internals of the action are created. In this example ROS_INFO is being published to let the user know that the action is executing.

In [ ]:
    // start executing the action
    for(int i=1; i<=goal->order; i++)
    {
      // check that preempt has not been requested by the client
      if (as_.isPreemptRequested() || !ros::ok())
      {
        ROS_INFO("%s: Preempted", action_name_.c_str());
        // set the action state to preempted
        as_.setPreempted();
        success = false;
        break;
      }
      feedback_.sequence.push_back(feedback_.sequence[i] + feedback_.sequence[i-1]);

An important component of an action server is the ability to allow an action client to request that the current goal execution be cancelled. When a client requests that the current goal be preempted the action server should cancel the goal, perform necessary clean-up, and call the function setPreempted(), which signals that the action has been preempted by user request. Setting the rate at which the action server checks for preemption requests is left to the implementor of the server.

In [ ]:
    // publish the feedback
      as_.publishFeedback(feedback_);
      // this sleep is not necessary, the sequence is computed at 1 Hz for demonstration purposes
      r.sleep();
    }

Here the Fibonacci sequence is put into the feedback variable and then published on the feedback channel provided by the action server. Then the action continues on looping and publishing feedback.

In [ ]:
    if(success)
    {
      result_.sequence = feedback_.sequence;
      ROS_INFO("%s: Succeeded", action_name_.c_str());
      // set the action state to succeeded
      as_.setSucceeded(result_);
    }
  }

Once the action has finished computing the Fibonacci sequence the action notifies the action client that the action is complete by setting succeeded.

In [ ]:
int main(int argc, char** argv)
{
  ros::init(argc, argv, "fibonacci");

  FibonacciAction fibonacci("fibonacci");
  ros::spin();

  return 0;
}

Finally the main function, creates the action and spins the node. The action will be running and waiting to receive goals.

<p style="background:#3B8F10;color:white;">**End Code Explanation C++ Program: {4.11a}**</p>

<p style="background:#EE9023;color:white;">**Exercise 4.12a: Check Fibonacci action msg structure**</p><br>
Check the structure of the Fibonacci.action message definition by visiting the <i>action</i> directory of the <i>actionlib_tutorials</i> package.

<p style="background:#EE9023;color:white;">**END Exercise 4.12a**</p>

<p style="background:#EE9023;color:white;">**Exercise 4.12b: Watch feedback and result topic messages output from the action server**</p><br>

Launch again the C++ code above <a href="#prg-4.11a">{4.11a}</a> to have the Fibonacci server running.
<br><br>
Then, execute the following commands in their corresponding WebShells.<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1: Echo the result topic</p>
</th>
</tr>
</table>

In [ ]:
rostopic echo /fibonacci_as/result

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2: Echo the feedback topic</p>
</th>
</tr>
</table>

In [ ]:
rostopic echo /fibonacci_as/feedback

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #3: Manually send the goal to your Fibonacci server, publishing directly to the topic (as you learned in the previous chapter)</p>
</th>
</tr>
</table>

In [ ]:
rostopic pub /fibonacci_as/goal actionlib_tutorials/FibonacciActionGoal [TAB][TAB]

<p style="background:#AE0202;color:white;">**Expected Result for Exercise 4.12b**</p><br>
After having called the action, the feedback topic should be publishing the feedback, and the result once the calculations are finished.

<br>

<p style="background:#3B8F10;color:white;">**Data for Exercise 4.12b**</p><br>
<ul>
<li>
You must be aware that the name of the messages (the class) used in the C++ code are called FibonacciGoal, FibonacciResult, and FibonacciFeedback, while the name of the messages used in the topics are called FibonacciActionGoal, FibonacciActionResult, and FibonacciActionFeedback. 
<br>
Do not worry about that, just bear it in mind and use it accordingly.
</li>
</ul>

<p style="background:#EE9023;color:white;">**END Exercise 4.12b**</p>

<p style="background:#EE9023;color:white;" id="ex-4-13">**Exercice 4.13: Create Package with Action Server that moves the Ardrone in the air, making a square**</p>
<br>
Create a package with an action server that makes the drone move in a square when called.
<br><br>
Call the action server through the topics and observe the result and feedback.
<br><br>
Base your code in the previous Fibonacci example <a href="#prg-4.11a">{4.11a}</a> and the client you did in Exercice 4.6 that moved the ardrone while taking pictures.
<br>

<p style="background:#AE0202;color:white;">**Expected Result for Exercice 4.13**</p><br>
The result must show the ardrone doing a square in the air when the action server is called, as shown in the animation beneath <a href="#fig-4.6">{Fig:4.6}</a>
<br>

<p style="background:#3B8F10;color:white;">**Data for Exercice 4.13**</p>
<ul>
<li>The size of the side of the square should be specified in the goal message as an integer.</li>
<li>The feedback should publish the current side (as a number) the robot is at while doing the square.</li>
<li>The result should publish the total number of seconds it took the drone to do the square</li>
<li>Use the <span style="color: orange"><i>Test.action</i></span> message for that action server. Use the shell command <span style="color: orange"><i>locate Test.action</i></span> to find where that message is defined. Then, analyze the different fields of the msg in order to learn how to use it in your action server.</li>
</ul>

<p style="background:#EE9023;color:white;" id="ex-4-13">**END Exercice 4.13**</p>




<figure>
  <img id="fig-4.6" src="img/drone.gif">
  <br>
   <center> <figcaption>Fig.4.6 - Ardrone moved through commands issed by an custom action server Ex 4.13</figcaption></center>
</figure>

## How to create your own action server message

<b>It is always recommended that you use the action messages already provided by ROS.</b> These can be found in the following ROS packages:

* actionlib
    
 * Test.action
 * TestRequest.action 
 * TwoInts.action
    
* actionlib_tutorials
    
 * Fibonacci.action
 * Averaging.action
    

<br>

However, it may happen that you need to create your own type. Let's learn how to do it.

To create your own custom action message you have to:

1.- Create an **action** directory within your package.
  
2.- Create your Name.action message file.
  
      
 * The Name of the action message file will determine later the name of the classes to be used in the **action server** and/or **action client**. ROS convention indicates that the name has to be camel-case.
        
 * Remember the Name.action file has to contain three parts, each part separated by three hyphens.  

In [ ]:
#goal
package_where_message_is/message_type goal_var_name
---
#result
package_where_message_is/message_type result_var_name
---
#feedback
package_where_message_is/message_type feedback_var_name

* If you do not need one part of the message (for example, you don't need to provide feedback), then you can leave that part empty. But you <b>must always specify the hyphen separtors</b>.     

3.- Modify the file CMakeLists.txt and the package.xml to include action message compilation. Read the detailed description below.

<div id="custom_compilation"></div>

## How to prepare CMakeLists.txt and package.xml files for custom action messages compilation

You have to edit two files in the package, in the same way that we explained for topics and services:

* CMakeLists.txt
* package.xml


### Modification of CMakeLists.txt

You will have to edit four functions inside CMakeLists.txt:

* <span class="ign_green"><a href="#find_package">find_package()</a></span>
* <span class="ign_green"><a href="#add_action_files">add_action_files()</a></span>
* <span class="ign_green"><a href="#generate_messages">generate_messages()</a></span>
* <span class="ign_green"><a href="#catkin_package">catkin_package()</a></span>



<b><span class="ign_green" id="find_package">find_package()</span></b>

All of the packages needed to COMPILE the messages of topic, services, and actions go here. 
<br>
In <i>package.xml</i>, you have to state them as built.

In [ ]:
find_package(catkin REQUIRED COMPONENTS
      # your packages are listed here
      actionlib_msgs
)

<b><span class="ign_green" id="add_action_files">add_action_files()</span></b>

This function will contain all of the action messages from this package (which are stored in the <b>action</b> folder) that need to be compiled.<br>
Place them beneath the FILES tag.

In [ ]:
add_action_files(
      FILES
      Name.action
)

<b><span class="ign_green" id="generate_messages">generate_messages()</span></b>

The packages needed for the action messages compilation are imported here. Write the same here as you wrote in the find_package.

In [ ]:
generate_messages(
      DEPENDENCIES
      actionlib_msgs 
      # Your packages go here
)

<b><span class="ign_green" id="catkin_package">catkin_package()</span></b>

State here all of the packages that will be needed by someone that executes something from your package.<br>
All of the packages stated here must be in the package.xml as run_depend.

In [ ]:
catkin_package(
      CATKIN_DEPENDS
      roscpp
      # Your package dependencies go here
)

Summarizing, You should end with a CMakeLists.txt similar to this:

In [ ]:
cmake_minimum_required(VERSION 2.8.3)
project(my_custom_action_msg_pkg)

## Find catkin macros and libraries
## if COMPONENTS list like find_package(catkin REQUIRED COMPONENTS xyz)
## is used, also find other catkin packages
find_package(catkin REQUIRED COMPONENTS
  std_msgs 
  actionlib_msgs
)

## Generate actions in the 'action' folder
add_action_files(
   FILES
   Name.action
 )

## Generate added messages and services with any dependencies listed here
generate_messages(
   DEPENDENCIES
   std_msgs actionlib_msgs
 )

catkin_package(
 CATKIN_DEPENDS roscpp
)

## Specify additional locations of header files
## Your package locations should be listed before other locations
# include_directories(include)
include_directories(
  ${catkin_INCLUDE_DIRS}
)

### Modification of package.xml:

1.- Add all of the packages needed to compile the messages.

If, for example, one of your variables in the .action file uses a message defined outside the std_msgs package, let's say **nav_msgs/Odometry**, you will need to import it. To do so, you would have to add as **build_depend** the **nav_msgs** package, adding the following line:

**<span class="ign_green">&lt;build_depend&gt;nav_msgs&lt;/build_depend&gt;</span>**

2.- On the other hand, if you need a package for the execution of the programs inside your package, you will have to import those packages as **run_depend**, adding the following line:

**<span class="ign_green">&lt;run_depend&gt;nav_msgs&lt;/run_depend&gt;</span>**

When you compile custom action messages, it's **mandatory** to add the **actionlib_msgs** as build_dependency.

**<span class="ign_red">&lt;build_depend&gt;actionlib_msgs&lt;/build_depend&gt;</span>**

When you use C++, it's **mandatory** to add the **roscpp** as **run_dependency**.

**<span class="ign_red">&lt;run_depend&gt;rospy&lt;/run_depend&gt;</span>**

This is due to the fact that the roscpp module is needed in order to run all of your C++ ROS code.

Summarizing, you should end with a package.xml file similar to this:

In [ ]:
<?xml version="1.0"?>
<package>
  <name>my_custom_action_msg_pkg</name>
  <version>0.0.0</version>
  <description>The my_custom_action_msg_pkg package</description>
  <maintainer email="user@todo.todo">user</maintainer>
  <license>TODO</license>

  <buildtool_depend>catkin</buildtool_depend>
  <build_depend>std_msgs</build_depend>
  <build_depend>actionlib_msgs</build_depend>
  
  <run_depend>roscpp</run_depend>

  <export>
  </export>
</package>

Finally, when everything is correctly set up, you just have to compile:

In [ ]:
roscd; cd ..
catkin_make
source devel/setup.bash
rosmsg list | grep Name

You will get an output to the last command, similar to this:

In [ ]:
my_custom_action_msg_pkg/NameAction
my_custom_action_msg_pkg/NameActionFeedback
my_custom_action_msg_pkg/NameActionGoal
my_custom_action_msg_pkg/NameActionResult
my_custom_action_msg_pkg/NameFeedback
my_custom_action_msg_pkg/NameGoal
my_custom_action_msg_pkg/NameResult

<p style="background:green;color:white;">**Note**</p><br>
Note that you haven't imported the <b>std_msgs</b> package anywhere. But you can use the messages declared there in your custom .actions. That's because this package forms part of the roscore file systems, so therefore, it's embedded in the compilation protocols, and no declaration of use is needed.

<p style="background:#EE9023;color:white;">**Exercise 4.14: Create a Package with an action server with custom action message to move ardone**</p>

* In the previous package, you created <a href="#ex-4-13">[Ex 4.13]</a>, create a new action server for the quadcopter
* The new action server will receive two words as a goal: UP or DOWN.
* When the action server receives the UP word, it will move the drone 1 meter up.
* When the action server receives the DOWN word, it will move the drone 1 meter down.
* As a feedback, it publishes once a second what action is taking place (going up or going down).
* When the action finishes, the result will return nothing.

<p style="background:#3B8F10;color:white;">**Data for Exercise 4.14**</p>

* You need to create a new action message with the specified values as <i>String</i>. This type can be imported from the <i>std_msgs</i> package.
* The result part of the action message will be empty.
* Since we are talking about a drone, you can specify Twist velocities in the three axes. You will need to do that in order to move the robot up and down.

<p style="background:#EE9023;color:white;">**END Exercise 4.14**</p>

## Additional information to learn more

ROS Actions: http://wiki.ros.org/actionlib

How actions work: http://wiki.ros.org/actionlib/DetailedDescription

## <p style="background:red;color:white;">Solutions</p>

Please Try to do it by yourself unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png"/>

Follow this link to open the solutions for the Actions Part 2:[Actions Part 2 Solutions](extra_files/unit4_basicROS_part2_solutions_cpp.ipynb)